In [1]:
from gensim.models import word2vec
import MeCab
import mojimoji
import pandas as pd
import numpy as np

In [2]:
def text_to_words(text,stop_word_pass='./Japanese.txt'):
#     stopword listを作る
    stopword_list=[]
#     stop_word_passをreadlineで全て読みstopword_listに格納
    with open(stop_word_pass,'r') as f:
        stopword_list = f.readlines()
#     内包表記でstripにて空白を削除してstopword_listに戻す
    stopword_list = [x.strip()for x in stopword_list if x.strip()]
    
#     形態素解析を始める
# MeCab.Tagger()で形態素解析をするための辞書を指定
    m = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
#     m.(tagger)parseで、指定した辞書を使用して、文字列やテキストファイルを形態素解析する
    m.parse('')
#     text=normalize_text(text)
    text=mojimoji.zen_to_han(text, kana=False)
    m_text = m.parse(text)
    basic_words = []
#     mecabの出力結果を単語ごとにリスト化
    m_text=m_text.split('\n')
    for row in m_text:
#     tab区切りで形態素、その品詞などの内容と分かれているので単語部のみ取得
        word =row.split("\t")[0]
#         最終行はEOS
        if word =="EOS":
            break
        else:
            pos=row.split('\t')[1]
            slice_ =pos.split(',')
#             品詞を取得する
            parts=slice_[0]
            if parts == '記号':
                continue
                
#             活用語の場合は活用指定のない原型を取得する
            elif slice_[0] in ('形容詞','動詞') and slice_[-3] not in stopword_list:
                basic_words.append(slice_[-3])
            
#             活用しない語についてはそのままの語を取得する
            elif slice_[0] == '名詞' and word not in stopword_list:
                basic_words.append(word)
    basic_words = ' '.join(basic_words)
    return basic_words

In [3]:
df_input = pd.read_csv('./data/kokkai.csv', header=0)
corpus = [x.split(' ') for x in \
          df_input['text'].map(text_to_words).values.tolist()]

In [4]:
corpus[1]

['議長',
 '山東昭子',
 '君',
 '国会',
 '本日',
 '召集',
 'する',
 'れる',
 '会議',
 '開く',
 '日程',
 '議席',
 '指定',
 '議長',
 '本',
 '院',
 '規則',
 '十四条',
 '規定',
 '皆さん',
 '議席',
 '議席',
 '指定',
 'いたす']

## モデル作成
モデル作成を行い、このモデルを保管する。 　
- size→次元数。
- min_count→何回以上出現した単語を対象にするか。
- window→前後どこまでを対象とするか。
- sg→1はski-gramそれ以外はCBOW

In [5]:
model = word2vec.Word2Vec(corpus, size=200, min_count=3, window=5, sg=1)
model.save('./data/kokkai_model.model')

### ベクトルの出力

In [11]:
model.wv['コロナ'].shape

(200,)

In [15]:
print(model.similarity('コロナ', 'アフター'))

0.76913106


<ipython-input-15-2b0c09197093>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  print(model.similarity('コロナ', 'アフター'))


In [17]:
similar_words = model.wv.most_similar(positive=['アフター'], topn=10) #top10
similar_words

[('ウイズコロナ', 0.9622336626052856),
 ('新たな日常', 0.8852411508560181),
 ('日本社会', 0.8806211352348328),
 ('傷む', 0.8753450512886047),
 ('経済政策', 0.8734216690063477),
 ('模索', 0.858197033405304),
 ('共存', 0.8571842908859253),
 ('局面', 0.8555092811584473),
 ('持ち直し', 0.8549432158470154),
 ('ピンチ', 0.8532471060752869)]